## Draw Contours on image with OpenCV


### Using Denoising and Ben's preprocessing
해당 전처리 방법 적용시 글자가 더욱 강조되어 contour에 도움이 됩니다

[kaggle Kuzushiji Recognition](https://www.kaggle.com/c/kuzushiji-recognition)에 참가한 [깃허브](https://github.com/knjcode/kaggle-kuzushiji-recognition-2019)를 참조했습니다


####  opencv imread 오류 관련 - window + 경로에 한글 포함시 주의사항 

* 이미지 불러올때 이미지를 cv2.imread로 바로 읽어 오지 않고   
* **np.fromfile 함수** 를 이용해서 바이너리 데이터를 넘파이 행렬로 읽는 과정을 추가    
그 다음 cv2.imdecode 함수로 복호화하여 opencv에서 이미지를 제대로 읽어오도록 함 

In [24]:
## preprocessing (denoising) 

import os
import sys
import numpy as np # linear algebra
import cv2 # image processing

from glob import glob
from multiprocessing import Pool

# Slightly modified copy of https://www.kaggle.com/hanmingliu/denoising-ben-s-preprocessing-better-clarity

# making sure result is reproducible
SEED = 2019
np.random.seed(SEED)

def read_image(image):
    '''
        Simply read a single image and convert it RGB in opencv given its filename.
    '''
    return cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)


def apply_ben_preprocessing(image):
    '''
        Apply Ben's preprocessing on a single image in opencv format
    '''
    return cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), 10), -4, 128)


def apply_denoising(image):
    '''
        Apply denoising on a single image given it in opencv format.
        Denoising is done using twice the recommended strength from opencv docs.
    '''
    return cv2.fastNlMeansDenoisingColored(image, None, 20, 20, 7, 21)

def denoise_and_bens_prepro_save(filepath):
    for name in target_files:
        filename = name.split('\\')[-1]
        print (filename)
        savename = filename.replace('.jpg', '.png')
        img = read_image('../hangul_input/필사본/'+name)
        prepro = apply_ben_preprocessing(img)
        after = apply_denoising(prepro)
        save_path = output_dir + savename
        cv2.imwrite(save_path, after)
        print(save_path)
        
def denoise_and_bens_prepro(img):
    prepro = apply_ben_preprocessing(img)
    after = apply_denoising(prepro)
    return after

In [21]:
## Do not use Denoising and Ben's preprocessing

input_path = 'ocr_input/h_inout/'
output_path = 'outdir/'
file_list = os.listdir(input_path)

for name in file_list:
    '''
    경로에 한글이 포함된 경우 아래 3줄 사용
    img_name = os.path.join(input_path, name)
    img_array = np.fromfile(img_name, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    '''
    img = cv2.imread(os.path.join(input_path, name))
    save_img = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
    cv2.THRESH_BINARY,15,2)
    
    kernel = np.ones((1,1), np.uint8) ## 튜플 숫자가 작을수록 얇아짐
    img_erode = cv2.erode(th2, kernel, iterations=1)
    img_erode = img_erode.astype(np.uint8) ## 이미지 처리 위해 uint8로 형변환 필요

    contours, hierarchy = cv2.findContours(th2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    list_area = []
    list_rect = []

    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)
        area = width * height
        if (img.shape[0]*0.01  < width < img.shape[0]*0.5) and ( img.shape[1]*0.01 < height < img.shape[1]*0.5): ## max_cont_area - 넣고싶은 숫자 (5000) ## &lt; 부등호(<) &gt; 부등호(>)
            list_rect.append((x, y, width, height))
            list_area.append(width * height)
            cv2.rectangle(save_img,(x,y),(x + width,y + height),(0,0,255),2)
        

    cv2.imwrite('{0}_contour.jpg'.format('outdir/'+name[0]), save_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [23]:
## Using Denoising and Ben's preprocessing


input_path = 'ocr_input/h_inout/'
output_path = 'outdir/'
file_list = os.listdir(input_path)

for name in file_list:
    '''
    경로에 한글이 포함된 경우 아래 3줄 사용
    img_name = os.path.join(input_path, name)
    img_array = np.fromfile(img_name, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    '''
    img = cv2.imread(os.path.join(input_path, name))
    save_img = img.copy()
    
    
    img = denoise_and_bens_prepro(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
    cv2.THRESH_BINARY,15,2)
    
    kernel = np.ones((1,1), np.uint8) ## 튜플 숫자가 작을수록 얇아짐
    img_erode = cv2.erode(th2, kernel, iterations=1)
    img_erode = img_erode.astype(np.uint8) ## 이미지 처리 위해 uint8로 형변환 필요

    contours, hierarchy = cv2.findContours(th2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    list_area = []
    list_rect = []

    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)
        area = width * height
        if (img.shape[0]*0.01  < width < img.shape[0]*0.5) and ( img.shape[1]*0.01 < height < img.shape[1]*0.5): ## max_cont_area - 넣고싶은 숫자 (5000) ## &lt; 부등호(<) &gt; 부등호(>)
            list_rect.append((x, y, width, height))
            list_area.append(width * height)
            cv2.rectangle(save_img,(x,y),(x + width,y + height),(0,0,255),2)
        

    cv2.imwrite('{0}_contour.jpg'.format('outdir/'+name[0]), save_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()